In [80]:
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import accuracy_score
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [46]:
# original dataset with specific index
# data = pd.read_csv('data/adult.data', na_values=["?"])
# target_dataset = pd.DataFrame(data.iloc[1:10001,].values)
# shadow_dataset = pd.DataFrame(data.iloc[15001:30001,].values)
# attack_test_nonmembers = pd.DataFrame(data.iloc[11001:14001,].values)
# attack_test_members = pd.DataFrame(data.iloc[2001:5001,].values)

In [81]:
def read_data(data_name):
    with np.load(DATA_PATH + data_name) as f:
        train_x, train_y, test_x, test_y = [f['arr_%d' % i] for i in range(len(f.files))]
    return train_x, train_y, test_x, test_y

In [82]:
def old_transform_data(dataset):
    x = dataset.iloc[:,0:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,-1] # label column
    

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=2

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data
            print(x.iloc[:,j])

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    
    y=to_categorical(y)
    
    return x, y, dim

In [83]:
def transform_data(dataset, is_synthetic):
    

    le = LabelEncoder()
    dataset[10] = le.fit_transform(dataset[10].astype('str'))

    # normalize the values
    x_range = [i for i in range(10)]
    #dataset[x_range] = dataset[x_range]/dataset[x_range].max()


    x = dataset[x_range].values
    y = dataset[10].values
        
    
    dim = x.shape[1]
    
    x=np.array(x)
    y=np.array(y)

    y=to_categorical(y)
    
    return x, y, dim

In [84]:
def load_target_data(dataset, test_ratio, is_synthetic):
    x, y, dim = transform_data(dataset, is_synthetic)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:7000,]
    testX = x[7000:,]
    trainY = y[0:7000,]
    testY = y[7000:,]
    return (trainX, trainY), (testX, testY), dim

In [85]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [86]:
def build_simple_dnn(n_class,dim, channel=0):
    model = Sequential()
    
    model.add(Dense(600, input_dim=dim))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(1024, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(512, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))

    model.add(Dense(256, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(128, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    #opt = SGD(lr=0.01, momentum=0.9)
    #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    act_layer=6
    
    return model, act_layer

In [87]:
def load_shadow_data(dataset, n_shadow, shadow_size, test_ratio, is_synthetic):
    x, y, _ = transform_data(dataset, is_synthetic)
    
    shadow_indices = np.arange(len(dataset))
    
   
    for i in range(n_shadow):
        shadow_i_indices = np.random.choice(shadow_indices, shadow_size, replace=False)
        shadow_i_x, shadow_i_y = x[shadow_i_indices], y[shadow_i_indices]
        trainX,testX, trainY, testY = train_test_split(shadow_i_x, shadow_i_y, test_size=test_ratio)
        print('shadow_i_trainX = ', len(trainX), 'shadow_i_trainY = ', len(trainY), 'shadow_i_testX = ', len(testX), 'shadow_i_testY = ', len(testY))
        
        np.savez(DATA_PATH + 'shadow_avila{}_data.npz'.format(i), trainX, trainY, testX, testY)

In [88]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]
    
    train_accuracy=[]
    test_accuracy=[]

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_avila{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

        model, act_layer = build_simple_mlp(n_class,dim, channel)
            
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))
        train_accuracy.append((train_acc * 100.0))
        test_accuracy.append((test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)
    shadow_accuracy = (train_accuracy, test_accuracy)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model, shadow_accuracy

In [89]:
def define_attack_model(n_class):
    model = Sequential()
    
    model.add(Dense(1))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.add(Dense(n_class, activation='softmax'))

    # compile model
    opt = SGD(learning_rate=0.0001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [90]:
def attack_mlp(pix,d):

    model = Sequential()
    model.add(Dense(64, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.1))

#     model.add(Dense(32))
#     model.add(Activation("tanh"))
#     model.add(Dropout(0.01))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=1
    
    return model, act_layer

In [91]:
def prep_attack_train_data(n_attack_data):

    attack_mem = pd.DataFrame(n_attack_data[0][0])
    attack_nmem = pd.DataFrame(n_attack_data[0][1])
    
    attack_mem_status = pd.DataFrame(n_attack_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(n_attack_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(n_attack_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(n_attack_data[2][1])
    real_class_nmem.columns = ["y"]

    memdf = pd.concat([attack_mem,attack_nmem],axis=0)
    memdf = memdf.reset_index(drop=True)

    memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
    memstatus = memstatus.reset_index(drop=True)

    realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
    realclass = realclass.reset_index(drop=True)

    attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return attack_df

In [92]:
def prep_validation_data(attack_test_data):

    attack_mem = pd.DataFrame(attack_test_data[0][0])
    attack_nmem = pd.DataFrame(attack_test_data[0][1])
    
    attack_mem_status = pd.DataFrame(attack_test_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(attack_test_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(attack_test_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(attack_test_data[2][1])
    real_class_nmem.columns = ["y"]
    
    mem_df = pd.concat([attack_mem,real_class_mem],axis=1)
    nmem_df = pd.concat([attack_nmem,real_class_nmem],axis=1)

#     memdf = pd.concat([attack_mem,attack_nmem],axis=0)
#     memdf = memdf.reset_index(drop=True)

#     memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
#     memstatus = memstatus.reset_index(drop=True)

#     realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
#     realclass = realclass.reset_index(drop=True)

#     attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return mem_df, nmem_df

In [93]:
def load_attack_test_data(members, nonmembers, is_synthetic):
    memberX, memberY, _ = transform_data(members, is_synthetic)
    
    nonmemberX, nonmemberY, _ = transform_data(nonmembers, is_synthetic)
    
    return memberX, memberY, nonmemberX, nonmemberY

In [94]:
def prety_print_result(mem, pred):
    tn, fp, fn, tp = confusion_matrix(mem, pred).ravel()
    print('TP: %d     FP: %d     FN: %d     TN: %d' % (tp, fp, fn, tn))
    if tp == fp == 0:
        print('PPV: 0\nAdvantage: 0')
    else:
        print('PPV: %.4f\nAdvantage: %.4f' % (tp / (tp + fp), tp / (tp + fn) - fp / (tn + fp)))

    return tp, fp, fn, tn, (tp / (tp + fp)), (tp / (tp + fn) - fp / (tn + fp)), ((tp+tn)/(tp+tn+fp+fn)),  (tp / (tp + fn))

In [95]:
def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    
    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)        
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
        
        d=1
        pix = full_cx_data.shape[1]
        classifier, _ = attack_mlp(pix,d)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        pred_y = classifier.predict(checkmem_prediction_vals[c_indices])
        print(pred_y)
        c_pred_y = np.argmax(pred_y, axis=1)
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])


    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall

In [96]:
def shokri_attack(attack_df, mem_validation, nmem_validation):
    
    predicted_membership, predicted_nmembership, true_membership, TP_idx, TN_idx  = [], [], [], [], []

    class_val = np.unique(attack_df['y'])
    ncval=attack_df.shape[1]-1
    
    for c_val in class_val:

        print(c_val)
        
        filter_rec_all = attack_df[(attack_df['y'] == c_val)]
        filter_rec_idx = np.array(filter_rec_all.index)
        
        attack_feat = filter_rec_all.iloc[:, 0:ncval]
        attack_class = filter_rec_all['membership']
             
        d=1
        pix = attack_feat.shape[1]
        
        attack_model, _ = attack_mlp(pix,d)
        
       
        history = attack_model.fit(attack_feat, attack_class, epochs=EPS, batch_size=32, verbose=0)
        
        mcval=mem_validation.shape[1]-1
        
        
        check_mem_feat = mem_validation[mem_validation['y']==c_val]
        check_nmem_feat = nmem_validation[nmem_validation['y']==c_val]
        
        if (len(check_mem_feat)!=0) and (len(check_nmem_feat)!=0):
        
            check_mem_feat_idx =  np.array(check_mem_feat.index)


            check_nmem_feat_idx =  np.array(check_nmem_feat.index)

            #print(check_nmem_feat_idx)
            #print(np.argmax(mpred,axis=1)==0)


            mpred = attack_model.predict(np.array(check_mem_feat))    
            predicted_membership.append(np.argmax(mpred,axis=1) )

            nmpred = attack_model.predict(np.array(check_nmem_feat))    
            predicted_nmembership.append(np.argmax(nmpred,axis=1) )        



            TP_idx.append(check_mem_feat_idx[np.where(np.argmax(mpred,axis=1)==1)[0]])

            TN_idx.append(check_nmem_feat_idx[np.where(np.argmax(nmpred,axis=1)==0)[0]])

    pred_members = np.array([item for sublist in predicted_membership for item in sublist])
    pred_nonmembers = np.array([item for sublist in predicted_nmembership for item in sublist])
    
    TP_idx_list = np.array([item for sublist in TP_idx for item in sublist])
    TN_idx_list = np.array([item for sublist in TN_idx for item in sublist])
    
    members=np.array(list(pred_members))
    nonmembers=np.array(list(pred_nonmembers))
    
    pred_membership = np.concatenate([members,nonmembers])
    ori_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
    return pred_membership, ori_membership, TP_idx_list, TN_idx_list

In [97]:
def train_target_model(target_dataset, per_class_sample, epoch, act_layer, n_class, is_synthetic, channel=0, verbose=0, test_ratio=0.3):
    
    (target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, test_ratio, is_synthetic)
    target_model,_ = build_simple_mlp(n_class,dim, channel)
    #get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
    history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
    score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
    _, train_acc = target_model.evaluate(target_trainX, target_trainY, verbose=VERBOSE)
    _, test_acc = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
    print('\n', "Target Train acc : ", (train_acc * 100.0),"Target Test acc : ", (test_acc * 100.0))
    #print('\n', 'Model test accuracy:', score[1])
    return target_model, dim

In [98]:
def prepare_attack_test_data(attack_test_members, attack_test_nonmembers, target_model, is_synthetic):
    members = []
    nonmembers = []

    memberX, memberY, nonmemberX, nonmemberY = load_attack_test_data(attack_test_members, attack_test_nonmembers, is_synthetic)

    # member
    target_model_member_pred = target_model.predict(memberX, batch_size=32)
    target_model_member_class = np.argmax(memberY, axis=1)
    target_model_member_pred = np.vstack(target_model_member_pred)
    #target_model_member_class = [item for sublist in target_model_member_class for item in sublist]
    members.append(np.ones(len(target_model_member_pred)))
    members = [item for sublist in members for item in sublist]


    # nonmember
    target_model_nonmember_pred = target_model.predict(nonmemberX, batch_size=32)
    target_model_nonmember_class = np.argmax(nonmemberY, axis=1)
    target_model_nonmember_pred = np.vstack(target_model_nonmember_pred)
    #target_model_nonmember_class = [item for sublist in target_model_nonmember_class for item in sublist]
    nonmembers.append(np.zeros(len(target_model_nonmember_pred)))
    nonmembers = [item for sublist in nonmembers for item in sublist]

    full_attack_test_pred_val = (target_model_member_pred, target_model_nonmember_pred)
    full_attack_test_mem_status = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    full_attack_test_class_status = (np.array(target_model_member_class),np.array(target_model_nonmember_class))

    print('\n pred', full_attack_test_pred_val)
    print('\n class', full_attack_test_class_status)
    print('\n mem status', full_attack_test_mem_status)

    attack_test_data = (full_attack_test_pred_val, full_attack_test_mem_status,full_attack_test_class_status)
    
    return attack_test_data

In [66]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [78]:
# original dataset with random index
data = pd.read_csv('data/avila-tr.txt', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 10000, replace = False)
df_rest = pd.read_csv('data/avila-ts.txt', na_values=["?"], header=None)
shadow_dataset = df_rest.sample(n = 7000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = 3000, replace = False)
attack_test_members =  target_dataset.iloc[:7000,:].sample(n = 3000, replace = False)

In [79]:
# trian target model
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 12
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, per_class_sample, EPS, act_layer, n_class, is_synthetic)


 Target Train acc :  96.12857103347778 Target Test acc :  90.73333144187927


In [99]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 5000

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(n_shadow_models, n_class, dim, channel)

shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_tes

In [100]:
#train attack model
attack_test_data = prepare_attack_test_data(attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

94/94 [==============================] - 0s 2ms/step

 pred (array([[9.7050071e-01, 2.3709588e-16, 5.3663268e-15, ..., 6.3308222e-16,
        2.0681337e-06, 1.8180718e-06],
       [8.8731172e-03, 3.8849543e-19, 9.2158836e-15, ..., 7.6288776e-15,
        1.0891775e-02, 4.2772226e-09],
       [6.6687059e-01, 2.3195396e-07, 2.8747135e-07, ..., 1.9356172e-07,
        2.9454189e-12, 4.5688250e-08],
       ...,
       [9.9968970e-01, 1.1148814e-15, 5.0343784e-05, ..., 2.1332906e-16,
        9.8672871e-08, 1.8458871e-14],
       [3.1852403e-01, 1.1128450e-11, 5.6032317e-09, ..., 1.0047440e-09,
        1.8475058e-06, 1.1344299e-06],
       [9.9997610e-01, 1.4525879e-16, 6.8545106e-22, ..., 5.8788912e-09,
        2.2700308e-06, 1.6626132e-09]], dtype=float32), array([[8.83870944e-02, 8.96694171e-16, 2.81694416e-12, ...,
        4.19103200e-17, 3.17756954e-09, 6.53733014e-06],
       [1.88080855e-02, 8.21492009e-18, 1.80693706e-15, ...,
        5.31825749e-17, 1.59894609e-09, 1.36780045e-08],
  

In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#

In [92]:
# synthetic dataset
target_dataset = pd.read_csv('data/adult_sds.csv', na_values=["?"], header=None)
shadow_dataset = pd.read_csv('data/adultODS10K_to_25K.csv', na_values=["?"], header=None)
attack_test_nonmembers = pd.read_csv('data/adultODS25K_to_30K.csv', na_values=["?"], header=None)
attack_test_members = pd.read_csv('data/adultODS1_to_5K.csv', na_values=["?"], header=None)

In [93]:
# trian target model
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
is_synthetic = True
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, per_class_sample, EPS, act_layer, n_class, is_synthetic)


 Target Train acc :  85.14285683631897 Target Test acc :  82.73333311080933


In [94]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 5000

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(n_shadow_models, n_class, dim, channel)

shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_tes

In [95]:
#train attack model
attack_test_data = prepare_attack_test_data(attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

157/157 [==============================] - 0s 1ms/step

 pred (array([[9.9985862e-01, 1.4152995e-04],
       [9.8442662e-01, 1.5573268e-02],
       [9.4646138e-01, 5.3538572e-02],
       ...,
       [9.9994189e-01, 5.8105015e-05],
       [9.8555994e-01, 1.4439992e-02],
       [9.9721175e-01, 2.7883006e-03]], dtype=float32), array([[9.9999624e-01, 3.7535829e-06],
       [2.1646561e-02, 9.7835344e-01],
       [5.6454796e-01, 4.3545207e-01],
       ...,
       [8.2621348e-01, 1.7378657e-01],
       [9.9999923e-01, 7.5641776e-07],
       [9.9825138e-01, 1.7485529e-03]], dtype=float32))

 class (array([0, 0, 0, ..., 0, 0, 0]), array([0, 1, 1, ..., 0, 0, 0]))

 mem status (array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([0, 0, 0, ..., 0, 0, 0], dtype=int32))
0
117/117 [==============================] - 0s 1ms/step
1
40/40 [==============================] - 0s 1ms/step
TP: 4811     FP: 4805     FN: 189     TN: 195
PPV: 0.5003
Advantage: 0.0012
Accuracy:  0.5006 Precision:  0.50031198003327

In [ ]:
# --------------------------------------------Synthetic Data-------------------------------------------------------------#

In [ ]:
# --------------------------------------------Overfitting Experiment-----------------------------------------------------#

In [266]:
# original dataset with random index
data = pd.read_csv('data/adult.data', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 10000, replace = False)
df_rest = data.loc[~data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 12000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = 5000, replace = False)
attack_test_members =  target_dataset.iloc[:7000,:].sample(n = 5000, replace = False)

In [325]:
def load_target_data(dataset, test_ratio, is_synthetic):
    x, y, dim = transform_data(dataset, is_synthetic)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:10,]
    testX = x[5000:,]
    trainY = y[0:10,]
    testY = y[5000:,]
    return (trainX, trainY), (testX, testY), dim

In [328]:
# trian target model
per_class_sample=5000
channel=0   
EPS=200
act_layer=2
n_class = 2
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, per_class_sample, EPS, act_layer, n_class, is_synthetic)

Model: "sequential_158"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_614 (Dense)           (None, 1024)              15360     
                                                                 
 activation_456 (Activation)  (None, 1024)             0         
                                                                 
 dense_615 (Dense)           (None, 512)               524800    
                                                                 
 activation_457 (Activation)  (None, 512)              0         
                                                                 
 dense_616 (Dense)           (None, 2)                 1026      
                                                                 
Total params: 541,186
Trainable params: 541,186
Non-trainable params: 0
_________________________________________________________________
None

 Target Train acc :  100.0 Target Test a

In [ ]:
# --------------------------------------------Overfitting Experiment-----------------------------------------------------#